In [1]:
import datetime as dt
from pathlib import Path
import os

import numpy as np
import pandas as pd


In [2]:
data_path = Path(os.getenv('HOME')+'/aiffel/yoochoose/data/') 
train_path = data_path / 'ratings.dat'

def load_data(data_path: Path, nrows=None):
    data = pd.read_csv(data_path, sep='::', header=None, usecols=[0, 1, 2, 3], dtype={0: np.int32, 1: np.int32, 2: np.int32}, nrows=nrows)
    data.columns = ['UserId', 'ItemId', 'Rating', 'Time']
    return data

data = load_data(train_path, None)
data.sort_values(['UserId', 'Time'], inplace=True)  # data를 id와 시간 순서로 정렬해줍니다.
data

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """


,UserId,ItemId,Rating,Time
31,1,3186,4,978300019
22,1,1270,5,978300055
27,1,1721,4,978300055
37,1,1022,5,978300055
24,1,2340,3,978300103
...,...,...,...,...
1000019,6040,2917,4,997454429
999988,6040,1921,4,997454464
1000172,6040,1784,3,997454464
1000167,6040,161,3,997454486


In [16]:
data.groupby('UserId').count()

,ItemId,Rating,Time
UserId,,,
1,53,53,53
2,129,129,129
3,51,51,51
4,21,21,21
5,198,198,198
...,...,...,...
6036,888,888,888
6037,202,202,202
6038,20,20,20


In [15]:
data.groupby('UserId').max() - data.groupby('UserId').min()

,ItemId,Rating,Time
UserId,,,
1,3407,2,524332
2,3872,4,2050
3,3764,4,1486
4,3442,4,358
5,3793,4,5513
...,...,...,...
6036,3572,4,45847
6037,3526,4,92843
6038,3316,4,10377
